In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [21]:
RACE=['White','Asian','Black']#,'Latino']
IMAGE_TEMPLATE=['_Male.png','_Female.png']
# Might need to translate to different langugage

In [22]:
def Session(idenList1,idenList2,diff=False):
    
    if diff:
        index=np.random.permutation(len(idenList1))
        iden1=idenList1[index[0]]
        iden2=idenList2[index[1]]
    else:
        iden1=idenList1[np.random.randint(len(idenList1))]
        iden2=idenList2[np.random.randint(len(idenList2))]
    
    indexImage=np.random.randint(0,high=len(IMAGE_TEMPLATE),size=(4,))
    imageFile=[iden1+IMAGE_TEMPLATE[indexImage[0]],
              iden2+IMAGE_TEMPLATE[indexImage[1]],
              iden1+IMAGE_TEMPLATE[indexImage[2]],
              iden2+IMAGE_TEMPLATE[indexImage[3]]]
    
    labels=[True,True,False,False]
    modelA=[True,True,True,False]
    modelB=[True,False,False,False]
    eqModelA='True Positive Rate Equilibrium'
    eqModelB='False Positive Rate Equilibrium'

    # Shuffle model A and model B
    if np.random.permutation(2)[0]:
        eqModelA='False Positive Rate Equilibrium'
        eqModelB='True Positive Rate Equilibrium'
        modelA=[True,False,False,False]
        modelB=[True,True,True,False]
        
    
    sessionnDF=pd.DataFrame(imageFile,columns=['imageFile'])
    sessionnDF['profile']=[iden1,iden2,iden1,iden2]
    sessionnDF['Label']=labels
    sessionnDF['ModelA_Prediction']=modelA
    sessionnDF['ModelB_Prediction']=modelB
    #sessionnDF=sessionnDF.sample(frac=1)

    return sessionnDF,eqModelA,eqModelB

In [23]:
def Generator(properties):
    Path=[]
    EqA=[]
    EqB=[]
    
    for i in range(len(properties)):
        
        # Generate csv file for the same identity
        listB=properties.copy()
        listB.remove(listB[i])
        
        sessionnDF,eqModelA,eqModelB = Session(properties[i:i+1],listB,diff=False)
        path='csv/'+properties[i]+'_Same.csv'
        sessionnDF.to_csv(path,index=False)       
        Path+=[path]
        EqA+=[eqModelA]
        EqB+=[eqModelB]
        
        # Generate csv file for different identity
        sessionnDF,eqModelA,eqModelB = Session(listB,listB,diff=True)
        path='csv/'+properties[i]+'_Diff.csv'
        sessionnDF.to_csv(path,index=False)                                                  
        Path+=[path]
        EqA+=[eqModelA]
        EqB+=[eqModelB]     
        
    csvfileProperties=pd.DataFrame(Path,columns=['Path'])
    csvfileProperties['ModelA']=EqA
    csvfileProperties['ModelB']=EqB
    return csvfileProperties

In [25]:
prop_RACE=Generator(RACE)

In [26]:
prop_RACE.to_csv('Csvfile_Properties.csv',index=False)